In [2]:
import pandas as pd
import numpy as np
import networkx as nx

### Funciones Auxiliares

In [130]:
# Funciones que dan formato a la cantidad de dígitos del ubigeo

def cantDigitosDis (x):
    return str(x).rjust(6, '0')

def cantDigitosPro (x):
    return str(x).rjust(4, '0')

def cantDigitosDep (x):
    return str(x).rjust(2, '0')

In [97]:
# Función que elimina atributos y transacciones no relevantes

def Eliminar_no_relevantes(dataset):
    
    # Obtener los atributos relevantes
    columnas_relevantes = [ 'amount_sol','agency_geoid', 'merchant_type', 'merchant_geoid']
    dataset_relevante = dataset.filter(columnas_relevantes, axis=1)

    # Eliminar transacciones con datos relevantes incompletos
    dataset_relevante.dropna(subset = columnas_relevantes, inplace=True)

    # Eliminar transacciones no turísticas
    tipo_turistico = ['Hoteles', 'Discotecas, penas, bares y pubs', 'Transporte Terrestre', 
                      'Agencias de Viajes','Transporte Aereo', 'Tiendas de artesanias y souveniers']
    serie = dataset_relevante.merchant_type.isin(tipo_turistico)
    dataset_relevante_filtro = dataset_relevante[serie]
    dataset_relevante_turistica = pd.DataFrame(dataset_relevante_filtro)
    
    return dataset_relevante_turistica

In [98]:
# Función que agrupa los datos según el nivel administrativo que indica el ubigeo de la agencia y el comercio

def Agrupar_por_nivel(dataset_relevante_turistica, agencia, comercio):
    
    dataset_grafo = dataset_relevante_turistica.groupby([agencia,
                                                         comercio])[['amount_sol']].agg({'amount_sol':'sum'})

    dataset_grafo.reset_index(inplace=True)
    dataset_grafo.columns = ['origin', 'destiny', 'amount']

    return dataset_grafo

In [140]:
# Función que genera un grafo según el conjunto de datos de entrada

def Generar_grafo(dataset_grafo):
    
    vertices = [x for x in dataset_grafo['origin'].unique()]
    vertices = np.concatenate((vertices, [x for x in dataset_grafo['destiny'].unique()]))
    vertices = list(set(vertices))

    aristas = [(row['origin'], row['destiny'], row['amount']) for index, row in dataset_grafo.iterrows()]

    G = nx.DiGraph()
    G.add_nodes_from(vertices)
    G.add_weighted_edges_from(aristas)
    
    return G

### Función Principal

In [131]:
def Preprocesamiento_y_Modelamiento(dataset):
    
    # Eliminar atributos y transacciones no relevantes
    dataset_relevante_turistica = Eliminar_no_relevantes(dataset)

    # Cambiar el tipo de dato del ubigeo
    columnas_ubigeo = ["agency_geoid", "merchant_geoid"]
    dataset_relevante_turistica[columnas_ubigeo] = dataset_relevante_turistica[columnas_ubigeo].astype(int)

    # Obtener ubigeo a nivel de departamentos y provincias
    dataset_relevante_turistica["agency_geoid_pro"] = dataset_relevante_turistica["agency_geoid"]//100
    dataset_relevante_turistica["agency_geoid_dep"] = dataset_relevante_turistica["agency_geoid"]//10000
    dataset_relevante_turistica["merchant_geoid_pro"] =dataset_relevante_turistica["merchant_geoid"]//100
    dataset_relevante_turistica["merchant_geoid_dep"] = dataset_relevante_turistica["merchant_geoid"]//10000

    # Homogenizar la cantidad de dígitos que se presenta como ubigeo
    dataset_relevante_turistica[['merchant_geoid']]=dataset_relevante_turistica['merchant_geoid'].map(cantDigitosDis)
    dataset_relevante_turistica[['merchant_geoid_pro']]=dataset_relevante_turistica['merchant_geoid_pro'].map(cantDigitosPro)
    dataset_relevante_turistica[['merchant_geoid_dep']]=dataset_relevante_turistica['merchant_geoid_dep'].map(cantDigitosDep)

    dataset_relevante_turistica[['agency_geoid']]=dataset_relevante_turistica['agency_geoid'].map(cantDigitosDis)
    dataset_relevante_turistica[['agency_geoid_pro']]=dataset_relevante_turistica['agency_geoid_pro'].map(cantDigitosPro)
    dataset_relevante_turistica[['agency_geoid_dep']]=dataset_relevante_turistica['agency_geoid_dep'].map(cantDigitosDep)
    
    # Agrupar datos por nivel administrativo
    dataset_grafo_dis = Agrupar_por_nivel(dataset_relevante_turistica,'agency_geoid','merchant_geoid' )
    dataset_grafo_pro = Agrupar_por_nivel(dataset_relevante_turistica,'agency_geoid_pro','merchant_geoid_pro' )
    dataset_grafo_dep = Agrupar_por_nivel(dataset_relevante_turistica,'agency_geoid_dep','merchant_geoid_dep' )

    # Generar los grafos por nivel administrativo
    G_dep = Generar_grafo(dataset_grafo_dep)
    G_pro = Generar_grafo(dataset_grafo_pro)
    G_dis = Generar_grafo(dataset_grafo_dis)
    
    return G_dep, G_pro, G_dis


### Pruebas

In [111]:
dataset = pd.read_csv("Transacciones_prueba.csv", ";")

In [122]:
print("Condición Inicial")
print("-----------------")
print("Cantidad de registros:", dataset.shape[0])
print("Cantidad de atributos:", dataset.shape[1], "\n")
print("Porcentaje de Vacíos:")
print(dataset.isna().sum()/len(dataset)*100)

Condición Inicial
-----------------
Cantidad de registros: 116800
Cantidad de atributos: 28 

Porcentaje de Vacíos:
id                      0.000000
client_id               0.000000
date                    0.000000
mcc                     0.004281
country_code            0.000000
amount_sol              0.003425
amount_usd              0.000000
nb_transaction          0.000000
client_age              0.000000
client_gender           0.000000
debit_type              0.000000
agency_id               0.000000
agency_geoid            0.008562
agency_departement      0.000000
agency_province         0.000000
agency_district         0.000000
agency_lima             0.000000
agency_region           0.000000
merchant_id             0.000000
merchant_name           0.000000
merchant_type           0.000000
merchant_geoid          0.014555
merchant_departement    0.000000
merchant_province       0.000000
merchant_district       0.016267
merchant_lon            0.000000
merchant_lat            0.

**Funciones Auxiliares**

1. Función "Eliminar_no_relevantes"

In [120]:
dataset_relevante_turistica = Eliminar_no_relevantes(dataset)

In [123]:
print("Condición Actual")
print("-----------------")
print("Cantidad de registros:", dataset_relevante_turistica.shape[0])
print("Cantidad de atributos:", dataset_relevante_turistica.shape[1], "\n")
print("Porcentaje de Vacíos:")
print(dataset_relevante_turistica.isna().sum()/len(dataset_relevante_turistica)*100)

Condición Actual
-----------------
Cantidad de registros: 84926
Cantidad de atributos: 4 

Porcentaje de Vacíos:
amount_sol        0.0
agency_geoid      0.0
merchant_type     0.0
merchant_geoid    0.0
dtype: float64


2. Función "Agrupar_por_nivel"

In [138]:
dataset_grafo_dis = Agrupar_por_nivel(dataset_relevante_turistica,'agency_geoid','merchant_geoid' )
dataset_grafo_dis.head()

,origin,destiny,amount
0,10101.0,10101.0,525.0
1,10101.0,20101.0,346.0
2,10101.0,21705.0,347.0
3,10101.0,30101.0,545.0
4,10101.0,40102.0,299.0


3. Función "Generar_grafo"

In [141]:
G_dis = Generar_grafo(dataset_grafo_dis)

In [142]:
nx.info(G_dis)

'DiGraph with 1805 nodes and 79966 edges'

**Función Principal**

In [143]:
G_dep, G_pro, G_dis = Preprocesamiento_y_Modelamiento(dataset)

In [144]:
G_dep['01']['15']

{'weight': 386172.0}

In [145]:
G_pro['0101']['1502']

{'weight': 3964.0}

In [146]:
G_dis['010102']['150201']

{'weight': 413.0}